### This notebook is an exemplar which demonstrates transferring files between a Box folder and Savio scratch while running interative processing on images using Photoscan (inside a Singularity container)

( tested with boxsdk (2.0.0a2) on python 3.5 kernel)
pip install -Iv boxsdk==2.0.0a2 


This software is available under the terms of the Educational Community License, Version 2.0 (ECL 2.0). This software is Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley").

The text of the ECL license is reproduced below.

Educational Community License, Version 2.0
*************************************
Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley")

Educational Community License, Version 2.0, April 2007

The Educational Community License version 2.0 ("ECL") consists of the
Apache 2.0 license, modified to change the scope of the patent grant in
section 3 to be specific to the needs of the education communities using
this license. The original Apache 2.0 license can be found at:[http://www.apache.org/licenses/LICENSE-2.0]

### Notebook configuration section
Set of target and source directories, script file names and other used as parameters in processing below.

In [2]:
runFolder = '/global/home/users/mmanning/'
realprojectFile = 'actestproject.psx'
projectFile = 'actestproject.psz'
projectfileid = ''
objFile = 'model.obj'
orthoFile = 'model.tiff'
singularitymountfolder = '/scratch/'

boxProjectFolder = 'actest2'
scratchImageDataDirectory = '/global/scratch/mmanning/actest/'

execScript = '/global/home/users/mmanning/actest/execscript.sh'
commandScript = '/global/home/users/mmanning/actest/commandscript.sh'

singularityContainerPath = '/global/scratch/mmanning/photoscan13.img'
SCP = '/global/scratch/mmanning/photoscan13.img'


### Box Authorization
function to store the oauth2 refresh token in a local file. This can be modified to use a keychain or other as required.

In [2]:
def store_tokens(access_token, refresh_token):
    
    """Callback for storing refresh tokens. (For now we ignore access tokens)."""
    with open('apptoken.cfg', 'w') as f:
     f.write(refresh_token.strip())

Oauth2 information is read from a local file with three lines, one line per parameter. 
The client id and client secret are defined in the Box application created for this notebook.  Create the application at the Box Developers site: https://berkeley.app.box.com/developers/services/edit/

The redirect uri can be any site that requires validation. Run the bootstrap notebook to create initial 
tokens that are then continually refreshed.

In [3]:
import os

CLIENT_ID = None
CLIENT_SECRET = None
REDIRECT_URI = None
os.chdir('/global/home/users/mmanning')
# Read app info from text file
with open('app.cfg', 'r') as app_cfg:
    CLIENT_ID = app_cfg.readline()
    CLIENT_SECRET = app_cfg.readline()
    REDIRECT_URI = app_cfg.readline()


The refresh token is read from a local file.
This token was created by running the bootstrap notebook which requires the user to validate
with CalNet Authentication Service credentials, then stores the returned auth and refresh tokens 
in the same config files.

In [4]:
REFRESH_TOKEN = None

# Read app info from text file
with open('apptoken.cfg', 'r') as apptoken_cfg:
    REFRESH_TOKEN = apptoken_cfg.readline()

__Perform autentication__ then create globus client
Verify client is working by retrieving the name of the users root folder in Box

In [5]:
from boxsdk import OAuth2
from boxsdk import Client

# Do OAuth2 authorization.
oauth = OAuth2(
    client_id=CLIENT_ID.strip(),
    client_secret=CLIENT_SECRET.strip(),
    refresh_token=REFRESH_TOKEN.strip(),
    store_tokens=store_tokens
)

client = Client(oauth)

root_folder = client.folder(folder_id='0').get()
print ("folder name: ", root_folder['name'] )

items = client.folder(folder_id='0').get_items(limit=100, offset=0)
#print ("items: ", items )



folder name:  All Files
items:  [<Box Folder - 16164064453 (401 gen 24 feb)>, <Box Folder - 13162986076 (actest)>, <Box Folder - 16461352722 (actest2)>, <Box Folder - 10920271405 (backupTest)>, <Box Folder - 12403779220 (cista Santa ISa)>, <Box Folder - 2998090751 (Connected Corridors Data)>, <Box Folder - 11943857315 (sdktest)>, <Box Folder - 11890577116 (TesseractExperiment)>, <Box Folder - 15846133956 (TesseractNotebook)>, <Box Folder - 12222643248 (test for Photoscan)>, <Box File - 92279466943 (.DS_Store)>, <Box File - 99364874757 (CNIP38 copy.pdf)>, <Box File - 99487811156 (LargeTestFile)>, <Box File - 91827878075 (Marine_CSV_sample.csv)>, <Box File - 97204968760 (moveNewToBox.sh)>, <Box File - 97209763161 (old1moveNewToBox.sh)>, <Box File - 98091014544 (Thumbs.db)>]


### Utility functions

__function to find folder id be folder name.__
Current SDK does not have a 'find by name' function so must loop thru all folders and look for match.

In [6]:
def find_folder_id(folder_name):
    print ('find_folder_id folder_name: ', folder_name)
    folderlist = client.search(query=folder_name, result_type='folder', limit=100, offset=0)
    
    print ('find_folder_id folderlist: ', folderlist)
    
    if len(folderlist) == 0:
        print('folder not found: ', folder_name)
        return 0
    else:
        for fldr in folderlist:
            if fldr['name'] == folder_name :
                return fldr['id']
            
        return 0

__Create command scripts template.__
Write out a execute file that includes the license install as the first step each time to enable any HPC node.
This is a temporary workaround until release 1.3 is installed on Savio and a license server is configured.
execscript - contains the license install as the first command and the command script as the second command.

In [7]:
import os, stat

execScriptTemplate = '/opt/photoscan-pro/photoscan.sh --activate TGN25-21RGK-UM9NG-UK49O-V55ZO\n\
/opt/photoscan-pro/photoscan.sh -r /global/home/users/mmanning/actest/commandscript.sh\n'

print (' exec script output: ', execScriptTemplate )

with open(execScript, 'w') as f:
    f.write(execScriptTemplate)
    
os.chmod(execScript,  stat.S_IRWXU)    

 exec script output:  /opt/photoscan-pro/photoscan.sh --activate TGN25-21RGK-UM9NG-UK49O-V55ZO
/opt/photoscan-pro/photoscan.sh -r /global/home/users/mmanning/actest/commandscript.sh



__Update the project file in the Box folder.__ 
Multiple times in the workflow we need to move the project file back to Box for manual editing on the desktop 
then retrieve the updated the file and continue the workflow. This utility funcation pushes the project 
file into the working directory on Savio.

In [8]:
import os
import shutil 
def update_project_file_in_box():
    projectfileid = 0
    newFolderId = find_folder_id(boxProjectFolder)
    
    projectfilelist = client.search(query=projectFile, result_type='file', limit=10, 
                                offset=0, ancestor_folders=[client.folder(folder_id=newFolderId)],
                                file_extensions=['psz'] )    
    print ("project file list: ", projectfilelist  )
    if len(projectfilelist) == 0 or len(projectfilelist) > 1: 
        print ("project file not found. " )
        return
    else:
        projectfileid = projectfilelist[0]['id']
        print ("project file id: ", projectpsz['id'] )
    
    update_file = client.file(file_id=projectfileid).get()

    # upload a new version of the project file
    print ('begin project file update.' )
    pszfile = update_file.update_contents(runFolder + projectFile)  
    print ('update psz result: ', pszfile )

__Retrieve the project file from the Box folder.__ 
Multiple times in the workflow we need to move the project file back to Box for manual editing on the desktop 
then retrieve the updated the file and continue the workflow. This utility funcation pulls the updated project 
file into the working directory on Savio.

In [9]:

import os
import shutil 
def retrieve_project_file():
    print ('current working directory: ', os.getcwd())
    #os.chdir(runFolder)

    newFolderId = find_folder_id(boxProjectFolder)
    tgtitems = client.folder(folder_id=newFolderId).get_items(limit=200, offset=0)

    # download the project file
    for tgtitem in tgtitems:
        if  not tgtitem['type'] == 'folder' and tgtitem['name'].endswith('.psz'):
            print('downloading: ', tgtitem['name'])
            imagecontent = client.file(file_id=tgtitem['id']).content()
            newfile = open(runFolder + tgtitem['name'], 'wb')
            newfile.write(imagecontent)
            newfile.close()
            print('download complete.')

### Retrieve the images from the Box folder.
currently the Box SDK does not have an option for finding a folder by name so if you are looking for a specific folder then you would need to loop thru all the items in the list below and do a name match. Once you find the folder and retrieve the id, you can save that id for subsequent runs. Another option is to get the id from the url in the web client, but approah below is more flexible for now. 

In [22]:
import os
import shutil 

print ('current working directory: ', os.getcwd())
os.chdir(scratchImageDataDirectory)

# test folder contents
items = client.folder(folder_id='0').get_items(limit=20, offset=0)
if type(items) is list:
    print ('number of files in top folder: ', len(items) )
    
    targetfolderId = ''
    for item in items:
        if item['type'] == 'folder':
            print('folder name: ', item['name'])
            if item['name'] == 'test for Photoscan':
                targetfolderId = item['id']
                print('targetfolderId: ', targetfolderId)
        
    if targetfolderId is not None:
        tgtitems = client.folder(folder_id=targetfolderId).get_items(limit=200, offset=0)
        if type(tgtitems) is list:
            print ('number of files in target folder: ', len(tgtitems) ) 
        
        # download all image files
        for tgtitem in tgtitems:
            if  not tgtitem['type'] == 'folder' and tgtitem['name'].endswith('.JPG'):
                print('dowmnloading: ', tgtitem['name'])
                imagecontent = client.file(file_id=tgtitem['id']).content()
                newfile = open(scratchImageDataDirectory + tgtitem['name'], 'wb')
                newfile.write(imagecontent)
                newfile.close()

current working directory:  /global/home/users/mmanning
number of files in top folder:  16
folder name:  401 gen 24 feb
folder name:  actest
folder name:  backupTest
folder name:  cista Santa ISa
folder name:  Connected Corridors Data
folder name:  sdktest
folder name:  TesseractExperiment
folder name:  TesseractNotebook
folder name:  test for Photoscan
targetfolderId:  12222643248
number of files in target folder:  40
dowmnloading:  IMG_5183.JPG
dowmnloading:  IMG_5184.JPG
dowmnloading:  IMG_5185.JPG
dowmnloading:  IMG_5186.JPG
dowmnloading:  IMG_5187.JPG
dowmnloading:  IMG_5188.JPG
dowmnloading:  IMG_5189.JPG
dowmnloading:  IMG_5190.JPG
dowmnloading:  IMG_5191.JPG
dowmnloading:  IMG_5192.JPG
dowmnloading:  IMG_5193.JPG
dowmnloading:  IMG_5194.JPG
dowmnloading:  IMG_5195.JPG
dowmnloading:  IMG_5196.JPG
dowmnloading:  IMG_5197.JPG
dowmnloading:  IMG_5198.JPG
dowmnloading:  IMG_5199.JPG
dowmnloading:  IMG_5200.JPG
dowmnloading:  IMG_5201.JPG
dowmnloading:  IMG_5204.JPG
dowmnloading:  IM

### Load the image set
run the first photoscan step: add the files
this cell created the script that will be executed by photoshop by writes the necessary commands into a text file.
mod the text in the step1Template string below as required.

clear docs about what needs to be set, can params be highlighted?


In [10]:
import os, stat

files = [ singularitymountfolder + f for f in os.listdir(scratchImageDataDirectory) ] 

template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
doc = PhotoScan.app.document \n\
chunk = PhotoScan.app.document.addChunk() \n\
chunk.addPhotos( {} ) \n\
doc.save(path=\"{}\", chunks = [doc.chunk])\n '

output = template.format(str(files), runFolder + projectFile)

with open(commandScript, 'w') as f:  
    f.write(output)

os.chmod(commandScript,  stat.S_IRWXU)

#### Execute the image load

In [11]:
    
out = !srun  --time=00:30:00 -A ac_scsguest --qos=savio_normal  --partition=savio  \
        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
        $singularityContainerPath  $execScript   
    
print (' image load output: ', out ) 


 image load output:  ['Activation successful', 'AddPhotos', 'SaveProject', 'saved project in 0.034856 sec', 'srun: error: n0009.savio1: task 0: Exited with exit code 1', 'srun: Terminating job step 1064919.0']


__Create a new folder in the base directory and upload the project (.psz) file in the current folder.__
If the folder for the project does not exisit, the folder is created.
If the folder already exists and contains a project file of the same name, the project file created in the previous step is NOT uploaded.

In [12]:
from boxsdk.exception import BoxAPIException

newFolderId = find_folder_id(boxProjectFolder)

if newFolderId == 0:
    newFolder = client.folder(folder_id='0').create_subfolder(boxProjectFolder)
    newFolderId = newFolder['id']
    print ("folder created" )
else: 
    print ("folder exists" )

print ("folder id: ", newFolderId )

upload_folder = client.folder(folder_id=newFolderId).get()


# check if project file already exists in that folder
projectfilelist = client.search(query=projectFile, result_type='file', limit=10, 
                                offset=0, ancestor_folders=[client.folder(folder_id=newFolderId)],
                                file_extensions=['psz'] )    
if len(projectfilelist) == 0 :
     # upload the project file
    projectpsz = upload_folder.upload(runFolder + projectFile)  
    print ("project file id: ", projectpsz['id'] )
else:
    print ("project file already exists")
    projectpsz = projectfilelist[0]['id']
    
projectfileid = projectpsz
print ("projectfileid : ", projectpsz )

find_folder_id folder_name:  actest2
find_folder_id folderlist:  []
folder not found:  actest2
folder created
folder id:  16567270680
project file id:  122511901070
projectfileid :  <Box File - 122511901070 (actestproject.psz)>


#### <span style="color:red">OFFLINE STEP:</span> User masks images in the project (psz) file locally using Photoscan on desktop.

### Build the dense cloud
__Retrieve the project file (psz) from Box as it may have been updated.__

In [13]:
retrieve_project_file()

current working directory:  /global/home/users/mmanning
find_folder_id folder_name:  actest2
find_folder_id folderlist:  []
folder not found:  actest2


Create the script for the second step: dense cloud build
this cell created the script that will be executed by photoshop by writes the necessary commands into a text file.
mod the text in the step2Template string below as required.

Options for PhotoScan.FilterMode: Depth filtering mode in [NoFiltering, MildFiltering, ModerateFiltering, AggressiveFiltering]

Options for PhotoScan.Quality: Dense point cloud quality in [UltraQuality, HighQuality, MediumQuality, LowQuality, LowestQuality]

In [14]:
import os, stat

template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.matchPhotos(accuracy=PhotoScan.HighAccuracy, preselection=PhotoScan.GenericPreselection) \n\
chunk.alignCameras(adaptive_fitting=True) \n\
chunk.buildDenseCloud(quality=PhotoScan.MediumQuality,  filter=PhotoScan.ModerateFiltering  ) \n\
doc.save() \n\
chunk = doc.chunk \n\
print( "stop time: ", time.time()) \n'

output = template.format(projectFile)

with open(commandScript, 'w') as f:  
    f.write(output)


#### Execute the dense cloud build.
savio_normal: 18.43333 minutes

savio_normal 11Jan: 15.65 mins/38 images

In [15]:

# GPUs
#out = !srun  --time=01:30:00 -A ac_scsguest  \
#        --partition=savio2_gpu --cpus-per-task=2 --gres=gpu:1 \
#        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
#        /global/scratch/mmanning/photoscan13.img   \
#        /global/home/users/mmanning/actest/execscript.sh

# Normal
#        --nodes=1 --cpus-per-task=1 --ntasks-per-node=20  \
out = !srun  --time=01:30:00 -A ac_scsguest  \
        --partition=savio --qos=savio_normal   \
        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
        $singularityContainerPath  $execScript
        

# Debug
#out = !srun  --time=00:30:00 -A ac_scsguest  \
#        --partition=savio --qos=savio_debug   \
#        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
#        /global/scratch/mmanning/photoscan13.img /global/home/users/mmanning/actest/execscript.sh    
    
    
#/opt/photoscan-pro/photoscan.sh -r /global/home/users/mmanning/actest/step_2.py
        
print (' step 2 output: ', out ) 

# clean up the script file
#os.remove(step2script)

 step 2 output:  ['Activation successful', 'start time:  1484287848.026246', 'LoadProject', 'loaded project in 0.018586 sec', 'MatchPhotos: accuracy = High, pair preselection = Generic, keypoint limit = 40000, tiepoint limit = 4000', 'photo 1: 39792 points', 'photo 2: 39861 points', 'photo 3: 39941 points', 'photo 4: 39931 points', 'photo 5: 39968 points', 'photo 6: 39943 points', 'photo 7: 39800 points', 'photo 8: 39999 points', 'photo 9: 39921 points', 'photo 10: 39812 points', 'photo 11: 39825 points', 'photo 12: 39948 points', 'photo 13: 39903 points', 'photo 14: 39990 points', 'photo 15: 39998 points', 'photo 16: 39853 points', 'photo 17: 39991 points', 'photo 18: 39861 points', 'photo 19: 39865 points', 'photo 20: 39841 points', 'photo 21: 39963 points', 'photo 22: 39783 points', 'photo 23: 39837 points', 'photo 24: 39787 points', 'photo 25: 39869 points', 'photo 26: 39847 points', 'photo 27: 39821 points', 'photo 28: 39974 points', 'photo 29: 39955 points', 'photo 30: 39850 poin

#### Move the resulting psz back to Box. 
Large file this time.

In [16]:
update_project_file_in_box()

find_folder_id folder_name:  actest2
find_folder_id folderlist:  [<Box Folder - 16567270680 (actest2)>]
project file list:  [<Box File - 122511901070 (actestproject.psz)>]
project file id:  122511901070
begin project file update.
update psz result:  <Box File - 122511901070>


#### <span style="color:red">OFFLINE STEP:</span> User cleans images in the project (psz) file locally using Photoscan on desktop.

### Build the mesh

__Retrieve the project file (psz) from Box as it may have been updated.__

In [17]:
retrieve_project_file()

current working directory:  /global/home/users/mmanning
find_folder_id folder_name:  actest2
find_folder_id folderlist:  [<Box Folder - 16567270680 (actest2)>]
downloading:  actestproject.psz
download complete.


__build the mesh__

Surface type in [Arbitrary, HeightField]

Face count in [LowFaceCount, MediumFaceCount, HighFaceCount]

Data source in [PointCloudData, DenseCloudData, ModelData, TiledModelData, ElevationData, OrthomosaicData]

In [18]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.buildModel(surface=PhotoScan.Arbitrary, source=PhotoScan.DenseCloudData, face_count=PhotoScan.HighFaceCount ) \n\
doc.save() \n\
chunk = doc.chunk \n\
print( "stop time: ", time.time()) \n'

output = template.format(projectFile)

with open(commandScript, 'w') as f:  
    f.write(output)


#### Execute the mesh build

In [19]:
    
out = !srun  --time=00:30:00 -A ac_scsguest --qos=savio_normal  --partition=savio  \
        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
        $singularityContainerPath  $execScript   
    
print ('  mesh build output: ', out ) 

  mesh build output:  ['Activation successful', 'start time:  1484291485.4682243', 'LoadProject', 'loaded project in 0.906071 sec', 'BuildModel: surface type = Arbitrary, source data = Dense cloud, face count = High, interpolation = Enabled', 'Grid size: 2499 x 1897 x 937', 'Tree depth: 12', 'Tree set in 30.0296s (4699733 points)', 'Leaves/Nodes: 31208815/35667217', 'Laplacian constraints set in 7.71011s', 'Depth[0/12]: 1', '\tEvaluated / Got / Solved in: 0 / 5.48363e-05 / 9.60827e-05', 'Depth[1/12]: 8', '\tEvaluated / Got / Solved in: 0 / 0.000277996 / 0.00965714', 'Depth[2/12]: 64', '\tEvaluated / Got / Solved in: 0 / 0.000825882 / 0.00664783', 'Depth[3/12]: 512', '\tEvaluated / Got / Solved in: 0 / 0.00103045 / 0.0110214', 'Depth[4/12]: 4096', '\tEvaluated / Got / Solved in: 0 / 0.00433302 / 0.0291336', 'Depth[5/12]: 32768', '\tEvaluated / Got / Solved in: 0 / 0.0403321 / 0.0508599', 'Depth[6/12]: 27080', '\tEvaluated / Got / Solved in: 0 / 0.0296288 / 0.0833011', 'Depth[7/12]: 9163

#### Move the resulting psz back to Box. 

In [20]:
update_project_file_in_box()

find_folder_id folder_name:  actest2
find_folder_id folderlist:  [<Box Folder - 16567270680 (actest2)>]
project file list:  [<Box File - 122511901070 (actestproject.psz)>]
project file id:  122511901070
begin project file update.
update psz result:  <Box File - 122511901070>


#### <span style="color:red">OFFLINE STEP:</span> User adds marker points in the project (psz) file locally using Photoscan on desktop.

### Build the texture and the orthomosaic

__Retrieve the project file (psz) from Box as it may have been updated.__

In [21]:
retrieve_project_file()

current working directory:  /global/home/users/mmanning
find_folder_id folder_name:  actest2
find_folder_id folderlist:  [<Box Folder - 16567270680 (actest2)>]
downloading:  actestproject.psz
download complete.


__build the texture and the orthomosaic__


Data source in [PointCloudData, DenseCloudData, ModelData, TiledModelData, ElevationData, OrthomosaicData]

To specify coordinate system add: 
chunk.crs = PhotoScan.CoordinateSystem("????")  \n\

In [3]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.buildUV(mapping=PhotoScan.GenericMapping) \n\
chunk.buildTexture( color_correction=True ) \n\
doc.save(runFolder + realprojectFile) \n\
doc.open(runFolder + realprojectFile) \n\
chunk = doc.chunk \n\
chunk.buildOrthomosaic( surface=PhotoScan.DataSource.ModelData ) \n\
doc.save() \n\
chunk = doc.chunk \n\
print( "stop time: ", time.time()) \n'

output = template.format(projectFile)

with open(commandScript, 'w') as f:  
    f.write(output)


#### Execute the texture and orthomosaic build

In [4]:
out = !srun  --time=00:50:00 -A ac_scsguest --qos=savio_normal  --partition=savio  \
        singularity exec -B  $scratchImageDataDirectory:$singularitymountfolder  \
        $singularityContainerPath  $execScript   
    
print ('  texture and orthomosaic build output: ', out ) 

  texture and orthomosaic build output:  ['Activation successful', 'start time:  1484368464.711834', 'LoadProject', 'loaded project in 1.3401 sec', 'BuildUV: blending mode = Generic, texture count = 1', 'Packing 633 charts...', 'BuildTexture: blending mode = Mosaic, texture size = 2048, enable color correction', 'collecting control points... collected 613065 points in 3.65725 sec', 'analyzing 35 images... *********************************** done in 2.88697 sec', 'estimating corrections... ', 'disabled 97 out of 514216 points (rms error 0.12287)', 'adjusting: xxx- 0.130218 -> 0.0887124', 'adjusting: xxxxx 0.121434 -> 0.082126', 'adjusting: xxxxxxxxxxxxxxxxxxxxxxxx 0.116343 -> 0.0734604', 'disabled 3313 out of 514216 points (rms error 0.0799492)', 'adjusting: xxxxxxxx 0.0840403 -> 0.0834751', 'adjusting: xxxxxxxxx 0.0774679 -> 0.0768954', 'adjusting: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 0.0690445 -> 0.0683913', 'disabled 4954 out of 514216 points (rms error 0.0761234)', 'adjusting: xx 0.0

#### Move the resulting psz back to Box. 

In [ ]:
update_project_file_in_box()

### Export results to Box

#### Export the orthomosaic format

In [11]:
import os
os.chdir(scratchImageDataDirectory)

template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.exportOrthomosaic(path=\"{}\", image_format=PhotoScan.ImageFormat.ImageFormatTIFF,)  \n' 

output = template.format(projectFile, scratchImageDataDirectory + orthoFile)

with open(commandScript, 'w') as f:
    f.write(output)


In [22]:
out = !srun  --time=01:30:00 -A ac_scsguest  \
        --partition=savio --qos=savio_normal   \
        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
        /global/scratch/mmanning/photoscan13.img /global/home/users/mmanning/actest/execscript.sh  
        
print (' Export the orthomosaic format output: ', out ) 

 step 3 output:  ['Activation successful', 'LoadProject', 'loaded project in 0.644578 sec', 'ExportModel', 'srun: error: n0101.savio1: task 0: Exited with exit code 1', 'srun: Terminating job step 1015000.0']


#### Move the resulting Orthomosaic file to Box. 

In [10]:
orthoUploaded = upload_folder.upload(scratchImageDataDirectory + orthoFile)  
print ("orthomosaic file id: ", orthoUploaded['id'] )

pdf file id:  118928041821


#### Generate OBJ format and load to Box
then download and view at: http://3dviewer.net/

In [74]:
import os
os.chdir(scratchImageDataDirectory)

template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.exportModel(path=\"{}\", format=PhotoScan.ModelFormat.ModelFormatOBJ)  \n' 

output = template.format(projectFile, scratchImageDataDirectory + objFile)

with open(commandScript, 'w') as f:
    f.write(output)


In [75]:
out = !srun  --time=01:30:00 -A ac_scsguest  \
        --partition=savio --qos=savio_normal   \
        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
        /global/scratch/mmanning/photoscan13.img /global/home/users/mmanning/actest/execscript.sh  
        
print (' Generate OBJ format output: ', out ) 

 Generate OBJ format output:  ['Activation successful', 'LoadProject', 'loaded project in 0.815927 sec', 'ExportModel', 'Traceback (most recent call last):', '  File "/global/home/users/mmanning/actest/commandscript.sh", line 6, in <module>', '    chunk.exportModel(path="/global/scratch/mmanning/actest/model.obj", format=PhotoScan.ModelFormat.ModelFormatOBJ)  ', 'RuntimeError: Null model', 'srun: error: n0022.savio1: task 0: Exited with exit code 1', 'srun: Terminating job step 1064340.0']


#### Move the resulting OBJ file to Box.

In [17]:
objUploaded = upload_folder.upload(scratchImageDataDirectory + objFile)  
print ("obj file id: ", objUploaded['id'] )

obj file id:  118928907887
